In [ ]:
import chess
import chess.uci
import chess.pgn
import os
import csv
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import math

In [ ]:
game_data_path = "data/game_data.pgn"
eval_csv_path = "data/stockfish_evals.csv"

pgns = open(game_data_path)
eval_csv = open(eval_csv_path)

sf_evals_csv = csv.reader(eval_csv, delimiter=',')

evals = []
for item in sf_evals_csv:
    item.pop(0)
    item = str(item)[2:-2].split()
    evals.append(item)
evals.pop(0)

    

In [ ]:


elo_and_performance = []

for i in range(5999):
    pw = []
    pb = []
    current_game = chess.pgn.read_game(pgns)
    try:
        pw.append(float(current_game.headers["WhiteElo"]))
    except:
        pw.append(0.0)
    try:
        pb.append(float(current_game.headers["BlackElo"]))
    except:
        pb.append(0.0)
    try:
        pw.append(float(evals[i][0]))
    except:
        pw.append(0.0) 
    for j in range (len(evals[i])):
        if j == 1:
            continue
        try:
            move_value = float(evals[i][j]) - float(evals[i][j-1])
        except:
            move_value = 0.0
        if j % 2 != 0: 
            pb.append(-move_value)
        else:
            pw.append(move_value)
    elo_and_performance.append(pw)
    elo_and_performance.append(pb)

elo_and_performance = array(elo_and_performance)

print(elo_and_performance.shape[0])


In [ ]:
data_points = []

for item in elo_and_performance:
    dp = []
    
    dp.append(item[0])
    
    arr = np.array(item[1:])
    
    average = np.mean(arr)
    dp.append(average)
    
    std = np.std(arr)
    dp.append(std)

    
    data_points.append(dp)

data_points = array(data_points)
np.set_printoptions(suppress=True)
print(data_points.shape)

In [ ]:
group_data = []

num_bins = 6

bins = np.linspace(1000, 2500, num_bins)

for bin in range (num_bins):
    group_data_point = []
    group_data_point.append(bins[bin])
    min=1000.0+(300*bin)
    group_count = 0
    group_mean = []
    group_std = []
    for point in data_points:
        if (point[0].item() > min) & (point[0].item() < min + 300):
            group_count +=1
            if not math.isnan(float(point[1])):
                group_mean.append(point[1])
            if not math.isnan(float(point[2])):
                group_std.append(point[2])
    group_mean = np.mean(array(group_mean))
    group_std = np.mean(array(group_std))

    if math.isnan(float(group_mean)):
        print(group_mean)
    if math.isnan(float(group_std)):
        print(group_std)

    group_data_point.append(group_mean)
    group_data_point.append(group_std)
    group_data.append(group_data_point)

data = array(group_data)


In [ ]:
 %pylab qt
N = data.shape[0]
print(N)
elos = [int(el[0]) for el in data]
means = [el[1] for el in data]
stds = [el[2] for el in data]

print(elos)
print(means)



### scaling for viewing
means = [el*3.5 for el in means]

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

ind = np.arange(N)  
width = 0.35 

fig, ax = plt.subplots()
rects1 = ax.bar(ind, means, width, color='seagreen')
rects2 = ax.bar(ind + width*.8, stds, width, color='darkslateblue')


ax.set_ylabel('Values (scaled for viewing)')
ax.set_xlabel('Elo bins ')

ax.set_title('Features of evaluated moves')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(elos)

plt.tick_params(left='off', labelleft='off')

ax.legend((rects1[0], rects2[0]), ('Average move strength', 'Standard deviation of move strength'))


plt.axhline(0, color='black')
plt.show()